In [7]:
import argparse
import os
import copy
os.environ['http_proxy'] = "http://10.176.52.116:7890"
os.environ['https_proxy'] = "http://10.176.52.116:7890"
os.environ['all_proxy'] = "socks5://10.176.52.116:7891"
import sys
# sys.path.append(('../'))
# sys.path.append(('../../'))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import json
from typing import Dict
import torch
from torch.nn import CrossEntropyLoss
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from task_vector import TaskVector
from itertools import chain
from watermarks.kgw.watermark_processor import WatermarkDetector
from watermarks.aar.aar_watermark import AarWatermarkDetector
from watermarks.watermark_types import WatermarkType
import logging
import torch.nn.functional as F

In [23]:
from collections import defaultdict
classified_dict = defaultdict(list)
for path in MLP_model_list:
    classified_dict[path[83:106]].append(path)
print(len(classified_dict['kgw-k1-gamma0.25-delta1']))
print(list(classified_dict.keys())) 

6
['kgw-k0-gamma0.25-delta1', 'kgw-k0-gamma0.25-delta2', 'kgw-k1-gamma0.25-delta2', 'kgw-k1-gamma0.25-delta1']


In [2]:
from datasets import load_dataset
import os
import copy
os.environ['http_proxy'] = "http://10.176.52.116:7890"
os.environ['https_proxy'] = "http://10.176.52.116:7890"
os.environ['all_proxy'] = "socks5://10.176.52.116:7891"
# Load the first 1% of the OpenWebText dataset
dataset = load_dataset("openwebtext", split='train[:1%]')


Generating train split:   0%|          | 0/8013769 [00:00<?, ? examples/s]

In [3]:
save_path = "/remote-home/share/data/openwebtext"  # Specify the path where you want to save the dataset
dataset.save_to_disk(save_path)


Saving the dataset (0/1 shards):   0%|          | 0/80138 [00:00<?, ? examples/s]

In [2]:
from datasets import load_dataset
import os
import copy
os.environ['http_proxy'] = "http://10.176.52.116:7890"
os.environ['https_proxy'] = "http://10.176.52.116:7890"
os.environ['all_proxy'] = "socks5://10.176.52.116:7891"
from torch.optim import AdamW
from transformers import AutoModelForCausalLM, AutoTokenizer, get_cosine_schedule_with_warmup,Cache
from datasets import load_dataset, Dataset, load_from_disk
from torch.utils.data import DataLoader
import sys
from tqdm import tqdm
sys.path.append(('../'))
sys.path.append(('../../'))
import logging
from task_vector import TaskVector
import logging
import copy

In [3]:
save_path = "/remote-home/share/data/openwebtext"
dataset = load_from_disk(save_path)

In [6]:
tested_model_name = "RohitSahoo/llama-2-7b-chat-hf-math-ft-V1"
tokenizer = AutoTokenizer.from_pretrained(tested_model_name)
tokenizer.pad_token = tokenizer.eos_token

In [14]:
model = AutoModelForCausalLM.from_pretrained("RohitSahoo/llama-2-7b-chat-hf-math-ft-V1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
batch_size = 4
optimizer = AdamW(model.parameters(), lr=1e-5, betas=(0.9, 0.999))
total_steps = int(2500 * (64/batch_size))
warmup_steps = int(0.1 * total_steps)  # 10% of total steps for warmup

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

In [7]:
batch_size = 4
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = DataLoader(tokenized_datasets, batch_size=batch_size, shuffle=True)

Map:   0%|          | 0/80138 [00:00<?, ? examples/s]

In [13]:
batch = next(iter(dataloader))
batch['attention_mask']

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

In [17]:
device = 'cpu'
index = 0
for batch in dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    # get_gpu_info()
    (f"{input_ids.shape}, {attention_mask.shape}")
    optimizer.zero_grad()
    # outputs = model(**inputs, labels=inputs['input_ids'])
    # print(batch.keys())
    # cache = Cache(model, batch['input_ids'], batch['attention_mask'])
    outputs = model(input_ids = input_ids, attention_mask = attention_mask, labels=input_ids)#, cache=cache)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    scheduler.step()
    print(f"{index}:{loss.item()}")
    index += 1
    # break

0:1.9419769048690796
1:4.125180721282959
2:2.830976724624634
3:2.038374662399292
4:5.609251976013184
5:2.373664140701294
6:4.295639991760254
7:4.111433506011963
8:2.560845375061035
9:3.267369508743286
10:2.0757975578308105
11:2.07499361038208
12:2.7399561405181885
13:1.9650331735610962
14:1.9647235870361328
15:3.525581121444702
16:4.107758045196533
17:2.586552858352661
18:1.6376299858093262
19:2.5431292057037354
20:2.0452449321746826
21:3.902618408203125
22:2.7822561264038086
23:2.009279489517212
24:2.370344877243042
25:2.616163969039917
26:3.718142032623291
27:1.7252116203308105
28:2.0578060150146484
29:3.540306806564331
30:3.650243043899536
31:1.9725979566574097
32:1.9312020540237427
33:2.0513861179351807
34:2.186934471130371
35:2.178136110305786
36:3.2234275341033936
37:2.2616355419158936
38:2.911768674850464
39:3.1385769844055176
40:3.766120195388794
41:3.315037488937378
42:2.5913455486297607
43:2.536370038986206
44:2.7551724910736084
45:2.076582908630371
46:2.4357078075408936
47:2

In [17]:
watermark_types = list(set([watermark_name[83:106] for watermark_name in MLP_model_list]))
print(watermark_types)

['kgw-k1-gamma0.25-delta1', 'kgw-k0-gamma0.25-delta1', 'kgw-k0-gamma0.25-delta2', 'kgw-k1-gamma0.25-delta2']


In [18]:
files

['/remote-home/miintern1/watermark-learnability/data/model_weights_2/model.layers.31_kgw-k0-gamma0.25-delta1_hidden_dim_16384_num_layers_2_best_model.pt',
 '/remote-home/miintern1/watermark-learnability/data/model_weights_2/model.layers.31_kgw-k0-gamma0.25-delta2_hidden_dim_8192_num_layers_2_best_model.pt',
 '/remote-home/miintern1/watermark-learnability/data/model_weights_2/model.layers.31_kgw-k1-gamma0.25-delta2_hidden_dim_16384_num_layers_3_best_model.pt',
 '/remote-home/miintern1/watermark-learnability/data/model_weights_2/model.layers.31_kgw-k0-gamma0.25-delta1_hidden_dim_8192_num_layers_2_best_model.pt',
 '/remote-home/miintern1/watermark-learnability/data/model_weights_2/model.layers.31_kgw-k0-gamma0.25-delta1_hidden_dim_8192_num_layers_3_best_model.pt',
 '/remote-home/miintern1/watermark-learnability/data/model_weights_2/model.layers.31_kgw-k0-gamma0.25-delta1_hidden_dim_16384_num_layers_1_best_model.pt',
 '/remote-home/miintern1/watermark-learnability/data/model_weights_2/mode

In [12]:
a = 'model.layers.31_kgw-k1-gamma0.25-delta1_hidden_dim_8192_num_layers_3_best_model'
b = 'hidden_dim_8192_num_layers_3'
p = '/remote-home/miintern1/watermark-learnability/data/model_weights_2/model.layers.31_kgw-k1-gamma0.25-delta1_hidden_dim_8192_num_layers_1_best_model.pt'
p[-42:-14]

'hidden_dim_8192_num_layers_1'

In [65]:
import json
with open('/remote-home/miintern1/watermark-learnability/data/c4/simulation_generation.json') as f:
    simulation_generation = json.load(f)
    print(simulation_generation.keys())
print(simulation_generation['kgw-k0-gamma0.25-delta2'].keys())
print(simulation_generation['vanilla_model'].keys())

dict_keys(['vanilla_model', 'kgw-k0-gamma0.25-delta1', 'kgw-k0-gamma0.25-delta2', 'kgw-k1-gamma0.25-delta2', 'kgw-k1-gamma0.25-delta1'])
dict_keys(['watermarked_model', 'hidden_dim_8192_num_layers_2', 'idden_dim_16384_num_layers_1', 'hidden_dim_8192_num_layers_1', 'idden_dim_16384_num_layers_2', 'hidden_dim_8192_num_layers_3', 'idden_dim_16384_num_layers_3'])
dict_keys(['full_output', 'truncate_prompt_output', 'watermark_scores', 'median_seq_rep_3', 'mean_seq_rep_3', 'list_seq_rep_3', 'total_rep_3'])


In [43]:
len(simulation_generation['kgw-k0-gamma0.25-delta2']['watermarked_model']['truncate_prompt_output'])
tokenizer

512

In [52]:
import numpy as np
from watermarks.kgw.watermark_processor import WatermarkDetector
from transformers import AutoTokenizer
vanilla_model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(vanilla_model_name)
def compute_p_value(samples, detector, type='kgw'):
    score_list = []
    for s in samples:
        score = detector.detect(s)
        score_list.append(score['p_value']) if type=='kgw' else score_list.append(score) 
    return score_list
detector = WatermarkDetector(
                    device='cpu',
                    tokenizer=tokenizer,
                    vocab=tokenizer.get_vocab().values(),
                    gamma=0.25,
                    seeding_scheme='simple_0',
                    normalizers=[],
                )


[0.795166233285417, 0.08362477687173493, 0.14235496852585539, 0.002617811328063631, 0.045622888154362884, 0.0007172067688006159, 0.023103250395920184, 2.39395138341366e-05, 8.026747830016304e-05, 0.09830732478526512, 0.004731966961401138, 0.0007172067688006159, 6.607832459974615e-06, 0.5873332091757439, 0.33187088109041213, 0.14235496852585539, 0.4562406987109025, 0.045622888154362884, 0.045622888154362884, 0.01085572810624082, 0.08362477687173493, 0.33187088109041213, 0.22526818027284895, 0.045622888154362884, 0.33187088109041213, 0.0007172067688006159, 0.33187088109041213, 0.4562406987109025, 0.004731966961401138, 0.055123360333705934, 0.0019130927836177925, 0.045622888154362884, 6.607832459974615e-06, 0.023103250395920184, 0.0007172067688006159, 0.004731966961401138, 0.2519188554772399, 0.9929537746788165, 0.0019130927836177925, 0.22526818027284895, 0.4562406987109025, 2.39395138341366e-05, 0.01085572810624082, 0.023103250395920184, 0.045622888154362884, 0.004731966961401138, 0.0108

In [58]:
simulation_generation['kgw-k0-gamma0.25-delta2'].keys()

dict_keys(['watermarked_model', 'hidden_dim_8192_num_layers_2', 'idden_dim_16384_num_layers_1', 'hidden_dim_8192_num_layers_1', 'idden_dim_16384_num_layers_2', 'hidden_dim_8192_num_layers_3', 'idden_dim_16384_num_layers_3'])

In [ ]:
import torch
import torch.nn as nn
import math

class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class TimeEmbedding(nn.Module):
    """
    TimeEmbedding模块将把整型t，以Transformer函数式位置编码的方式，映射成向量，
    其shape为(batch_size, time_channel)
    """

    def __init__(self, n_channels: int):
        """
        Params:
            n_channels：即time_channel
        """
        super().__init__()
        self.n_channels = n_channels
        self.lin1 = nn.Linear(self.n_channels // 4, self.n_channels)
        self.act = Swish()
        self.lin2 = nn.Linear(self.n_channels, self.n_channels)

    def forward(self, t: torch.Tensor):
        """
        Params:
            t: 维度（batch_size），整型时刻t
        """
        # 以下转换方法和Transformer的位置编码一致
        # 【强烈建议大家动手跑一遍，打印出每一个步骤的结果和尺寸，更方便理解】
        half_dim = self.n_channels // 8
        print(f"half_dim: {half_dim}")

        emb = math.log(10_000) / (half_dim - 1)
        print(f"emb (log scale factor): {emb}")

        emb = torch.exp(torch.arange(half_dim, device=t.device) * -emb)
        print(f"emb (after exp): {emb}")

        emb = t[:, None] * emb[None, :]
        print(f"emb (after scaling with t): {emb}")

        emb = torch.cat((emb.sin(), emb.cos()), dim=1)
        print(f"emb (after sin/cos): {emb}")

        # Transform with the MLP
        emb = self.act(self.lin1(emb))
        print(f"emb (after lin1 and act): {emb}")

        emb = self.lin2(emb)
        print(f"emb (after lin2): {emb}")

        # 输出维度(batch_size, time_channels)
        return emb

# Example test code
if __name__ == "__main__":
    batch_size = 2
    time_channels = 32
    time_embedding = TimeEmbedding(time_channels)

    t = torch.tensor([1, 2], dtype=torch.float32)  # Example batch of time steps
    print(f"Input t: {t}")
    output = time_embedding(t)
    print(f"Output: {output}")


In [69]:
quantiles = [0.25, 0.5, 0.75] 
print(f"Quantile values: {quantiles}")
for watermark_type, watermark_sd in simulation_generation.items():
    print(watermark_type)
    detector = WatermarkDetector(
                    device='cpu',
                    tokenizer=tokenizer,
                    vocab=tokenizer.get_vocab().values(),
                    gamma=0.25,
                    seeding_scheme='simple_0',
                    normalizers=[],
                )
    # print(watermark_sd.keys())
    if watermark_type == 'vanilla_model':
        output_scores = compute_p_value(watermark_sd['truncate_prompt_output'], detector, type='kgw')
        quantile_values = np.quantile(output_scores, quantiles)
        print(quantile_values)
        print('-'*100)
    else:
        for model_name, generations in watermark_sd.items():
            output_scores = compute_p_value(generations['truncate_prompt_output'], detector, type='kgw')
            output_scores = np.array(output_scores)
            print(model_name)
            quantile_values = np.quantile(output_scores, quantiles)
            print(quantile_values)
            print('-'*100)
            # print(np.median(output_scores))
            # break
    print('='*100)

Quantile values: [0.25, 0.5, 0.75]
vanilla_model
[0.22526818 0.4562407  0.71152611]
----------------------------------------------------------------------------------------------------
kgw-k0-gamma0.25-delta1
watermarked_model
[0.00424435 0.03717802 0.14235497]
----------------------------------------------------------------------------------------------------
num_layers_1_hidden_dim_16384
[0.04562289 0.16308327 0.4562407 ]
----------------------------------------------------------------------------------------------------
idden_dim_16384_num_layers_2
[0.04562289 0.22526818 0.4562407 ]
----------------------------------------------------------------------------------------------------
hidden_dim_8192_num_layers_2
[0.049921   0.22526818 0.4562407 ]
----------------------------------------------------------------------------------------------------
hidden_dim_8192_num_layers_3
[0.04562289 0.22526818 0.4562407 ]
-----------------------------------------------------------------------------

KeyboardInterrupt: 

In [64]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

model_name = "RohitSahoo/llama-2-7b-chat-hf-math-ft-V1"
import os
import copy
os.environ['http_proxy'] = "http://10.176.52.116:7890"
os.environ['https_proxy'] = "http://10.176.52.116:7890"
os.environ['all_proxy'] = "socks5://10.176.52.116:7891"

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # use the gpu
    device_map="cpu"
)

# don't use the cache
model.config.use_cache = False

# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


text = '''Reward Modelling Phase: In the second phase the SFT model is prompted with prompts x to
produce pairs of answers (y1, y2 ) ∼ π SFT (y | x). These are then presented to human labelers
who express preferences for one answer, denoted as yw ≻ yl | x where yw and yl denotes the
preferred and dispreferred completion amongst (y1 , y2 ) respectively. The preferences are assumed
to be generated by some latent reward model r∗ (y, x), which we do not have access to. There are a
number of approaches used to model preferences, the Bradley-Terry (BT) [5] model being a popular
choice (although more general Plackett-Luce ranking models [30, 21] are also compatible with the
framework if we have access to several ranked answers). The BT model stipulates that the human
preference distribution p∗ can be written as:
p∗ (y1 ≻ y2 | x) =
exp (r∗ (x, y1 ))
.
exp (r∗
(x, y1 )) + exp (r ∗ (x, y2))
(1)
Assuming access to a static dataset of comparisons D = 
x(i), yw (i) , yl (i) N
i=1 sampled from p∗ , we
can parametrize a reward model rφ (x, y) and estimate the parameters via maximum likelihood.
Framing the problem as a binary classification we have the negative log-likelihood loss:
LR(rφ , D) = −E(x,yw ,yl )∼D 
log σ(rφ (x, yw ) − rφ (x, yl )).

Please explain the math behind this paper by explaining all the variables'''


import time
start = time.time()

logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1500)
result = pipe(f"<s>[INST] {text} [/INST]")
print(result[0]['generated_text'])

end = time.time()
print(end - start)


2024-08-05 13:24:10.309529: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 13:24:13.999439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 13:24:14.974190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 13:24:15.439064: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 13:24:17.528727: I tensorflow/core/platform/cpu_feature_guar

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<s>[INST] Reward Modelling Phase: In the second phase the SFT model is prompted with prompts x to
produce pairs of answers (y1, y2 ) ∼ π SFT (y | x). These are then presented to human labelers
who express preferences for one answer, denoted as yw ≻ yl | x where yw and yl denotes the
preferred and dispreferred completion amongst (y1 , y2 ) respectively. The preferences are assumed
to be generated by some latent reward model r∗ (y, x), which we do not have access to. There are a
number of approaches used to model preferences, the Bradley-Terry (BT) [5] model being a popular
choice (although more general Plackett-Luce ranking models [30, 21] are also compatible with the
framework if we have access to several ranked answers). The BT model stipulates that the human
preference distribution p∗ can be written as:
p∗ (y1 ≻ y2 | x) =
exp (r∗ (x, y1 ))
.
exp (r∗
(x, y1 )) + exp (r ∗ (x, y2))
(1)
Assuming access to a static dataset of comparisons D = 
x(i), yw (i) , yl (i) N
i=1 sampled from p∗ , 

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

class LogitsDataset(Dataset):
    def __init__(self, input_logits, target_logits):
        self.input_logits = input_logits
        self.target_logits = target_logits

    def __len__(self):
        return len(self.input_logits)

    def __getitem__(self, idx):
        return self.input_logits[idx], self.target_logits[idx]

final_result_folder = "/remote-home/miintern1/watermark-learnability/data/c4/"
watermark_residuals = torch.load(final_result_folder + "watermark_residuals.pt", map_location=torch.device('cpu'))
vanilla_residuals = torch.load(final_result_folder + "vanilla_residuals.pt", map_location=torch.device('cpu'))

In [28]:
watermark_configs = {
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k0-gamma0.25-delta1":{"type": "kgw", "k": 0, "gamma": 0.25, "delta": 1.0, "seeding_scheme": "simple_0", "kgw_device": "cpu"},
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k0-gamma0.25-delta2":{"type": "kgw", "k": 0, "gamma": 0.25, "delta": 2.0, "seeding_scheme": "simple_0", "kgw_device": "cpu"},
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta1":{"type": "kgw", "k": 1, "gamma": 0.25, "delta": 1.0, "seeding_scheme": "simple_1", "kgw_device": "cpu"},
     "cygu/llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta2":{"type": "kgw", "k": 1, "gamma": 0.25, "delta": 2.0, "seeding_scheme": "simple_1", "kgw_device": "cpu"},
}
batch_size = 64
model_dimension = 4096
hidden_dimension = model_dimension * 4
num_layers = 4
analyze_layers = list(vanilla_residuals.keys())
print(analyze_layers)
target_layer = 'model.layers.31'
target_watermark =  "cygu/llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta2"

dataset_dict = dict()
dataloader_dict = dict()
for layer in analyze_layers:
    dataset_dict[layer] = dict()
    dataloader_dict[layer] = dict()
    for watermark_name, _ in watermark_configs.items():
        dataset_dict[layer][watermark_name] = LogitsDataset(vanilla_residuals[layer], watermark_residuals[watermark_name][layer])
        dataloader_dict[layer][watermark_name] = DataLoader(dataset_dict[layer][watermark_name], batch_size=batch_size, shuffle=True)

dataloaders = dataloader_dict[target_layer][target_watermark]

['model.layers.28', 'model.layers.29', 'model.layers.30', 'model.layers.31']


In [32]:
class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super(ResidualBlock, self).__init__()
        self.fc = nn.Linear(dim, dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc(x)
        out = self.relu(out)
        out = out + x 
        return out

class TransformModel(nn.Module):
    def __init__(self, num_layers=4, input_dim=1024, hidden_dim=500, output_dim=300):
        super(TransformModel, self).__init__()
        
        self.layers = nn.ModuleList()
        
        self.layers.append(nn.Linear(input_dim, hidden_dim))
        
        for _ in range(num_layers - 2):
            self.layers.append(ResidualBlock(hidden_dim))

        self.layers.append(nn.Linear(hidden_dim, output_dim))

    def forward(self, x):
        for i in range(len(self.layers)):
            x = self.layers[i](x)
        return x

# Training function
def train_model(model, dataloader, num_epochs=10, learning_rate=0.001):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for inputs, targets in progress_bar:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            progress_bar.set_postfix(loss=running_loss/len(dataloader))
            break
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")
        break
    print("Training complete.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformModel(num_layers=4, input_dim=4096, hidden_dim=4096*4, output_dim=4096)
train_model(model, dataloader, num_epochs=10, learning_rate=0.001)


Epoch 1/10:   0%|          | 0/217 [00:03<?, ?it/s, loss=0.0164]


Epoch [1/10], Loss: 0.0164
Training complete.


In [49]:
train_ratio = 0.9
dataset_dict = dict()
train_dataloader_dict = dict()
eval_dataloader_dict = dict()
for layer in analyze_layers:
    dataset_dict[layer] = dict()
    train_dataloader_dict[layer] = dict()
    eval_dataloader_dict[layer] = dict()
    for watermark_name, _ in watermark_configs.items():
        dataset_dict[layer][watermark_name] = LogitsDataset(vanilla_residuals[layer], watermark_residuals[watermark_name][layer])
        train_dataset, test_dataset = torch.utils.data.random_split(dataset_dict[layer][watermark_name], [int(train_ratio*len(dataset_dict[layer][watermark_name])), len(dataset_dict[layer][watermark_name]) - int(train_ratio*len(dataset_dict[layer][watermark_name]))])
        train_dataloader_dict[layer][watermark_name] = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        eval_dataloader_dict[layer][watermark_name] = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

dataloader = train_dataloader_dict[target_layer][target_watermark]
eval_dataloader = eval_dataloader_dict[target_layer][target_watermark]
print(len(eval_dataloader))
len(dataloader)



def evaluate_model(model, dataloader):
    model = model.to(device)
    model.eval()
    criterion = nn.MSELoss()
    
    num_layers = len(model.layers)
    hidden_dimension = model.layers[0].out_features
    hyperparameter_name = f"num_layers_{num_layers}_hidden_dim_{hidden_dimension}"
    # writer = SummaryWriter(log_dir=f'runs/{hyperparameter_name}/evaluation')

    running_loss = 0.0
    # progress_bar = tqdm(dataloader, desc="Evaluating")
    
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(dataloader):
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item()
            # progress_bar.set_postfix(loss=running_loss/(i+1))
    
    eval_loss = running_loss / len(dataloader)
    # writer.add_scalar('Evaluation Loss', eval_loss, 0)
    # writer.close()
    return eval_loss

eval_loss = evaluate_model(model, eval_dataloader)
print(f"Evaluation loss: {eval_loss:.4f}")


22
Evaluation loss: 626.3929


In [ ]:
def train_model(model, dataloader, num_epochs=10, learning_rate=0.001):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for inputs, targets in progress_bar:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            progress_bar.set_postfix(loss=running_loss/len(dataloader))
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

    print("Training complete.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformModel(num_layers=4, input_dim=4096, hidden_dim=4096*4, output_dim=4096)
train_model(model, dataloader, num_epochs=10, learning_rate=0.001)


In [5]:
import os
import glob
def extract_files(folder_path):
    files = glob.glob(os.path.join(folder_path, '*'))
    files = [f for f in files if os.path.isfile(f)]
    return files

from collections import defaultdict
folder_path = '/remote-home/miintern1/watermark-learnability/data/model_weights_2'
MLP_model_list = extract_files(folder_path)
MLP_model_path_dict = defaultdict(list)
for path in MLP_model_list:
    MLP_model_path_dict[path[83:106]].append(path)
watermark_list = list(MLP_model_path_dict.keys())

In [53]:
import json
with open('/remote-home/miintern1/watermark-learnability/data/c4/simulation_generation.json') as f:
    simulation_generation = json.load(f)
print(simulation_generation.keys())
simulation_generation['kgw-k0-gamma0.25-delta1'].keys()

dict_keys(['vanilla_model'])


KeyError: 'kgw-k0-gamma0.25-delta1'

dict_keys(['kgw-k0-gamma0.25-delta1', 'kgw-k0-gamma0.25-delta2', 'kgw-k1-gamma0.25-delta2'])


dict_keys(['watermarked_model', 'num_layers_1_hidden_dim_16384', 'num_layers_1_hidden_dim_8192', 'num_layers_2_hidden_dim_16384'])

In [5]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)
import os
os.environ['http_proxy'] = "http://10.176.52.116:7890"
os.environ['https_proxy'] = "http://10.176.52.116:7890"
os.environ['all_proxy'] = "socks5://10.176.52.116:7891"
import sys

model_name = "RohitSahoo/llama-2-7b-chat-hf-math-ft-V1"


# load the quantized settings, we're doing 4 bit quantization
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_use_double_quant=False,
# )

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    # use the gpu
    device_map='cpu'
)

# don't use the cache
model.config.use_cache = False

# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:


text = '''Reward Modelling Phase: In the second phase the SFT model is prompted with prompts x to
produce pairs of answers (y1, y2 ) ∼ π SFT (y | x). These are then presented to human labelers
who express preferences for one answer, denoted as yw ≻ yl | x where yw and yl denotes the
preferred and dispreferred completion amongst (y1 , y2 ) respectively. The preferences are assumed
to be generated by some latent reward model r∗ (y, x), which we do not have access to. There are a
number of approaches used to model preferences, the Bradley-Terry (BT) [5] model being a popular
choice (although more general Plackett-Luce ranking models [30, 21] are also compatible with the
framework if we have access to several ranked answers). The BT model stipulates that the human
preference distribution p∗ can be written as:
p∗ (y1 ≻ y2 | x) =
exp (r∗ (x, y1 ))
.
exp (r∗
(x, y1 )) + exp (r ∗ (x, y2))
(1)
Assuming access to a static dataset of comparisons D = x(i), yw (i) , yl (i) N
i=1 sampled from p∗ , we
can parametrize a reward model rφ (x, y) and estimate the parameters via maximum likelihood.
Framing the problem as a binary classification we have the negative log-likelihood loss:
LR(rφ , D) = −E(x,yw ,yl )∼D 
log σ(rφ (x, yw ) − rφ (x, yl )).

Please explain the math behind this paper by explaining all the variables'''


import time
start = time.time()

logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1500)
result = pipe(f"<s>[INST] {text} [/INST]")
print(result[0]['generated_text'])

end = time.time()
print(end - start)


In [ ]:
from tqdm import tqdm
block_size = 512
batch_size = 16
all_input_ids = []
all_attention_masks = []
hook_layer = range(28,32)
for example in tqdm(dataset):
    text = example["text"]
    # Tokenize the text
    tokenized_text = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=block_size)
    
    # Add the tokenized text to the lists
    all_input_ids.append(tokenized_text['input_ids'].squeeze().tolist())
    all_attention_masks.append(tokenized_text['attention_mask'].squeeze().tolist())


NameError: name 'tqdm' is not defined

In [ ]:
all_input_ids = list(chain(*all_input_ids))
all_attention_masks = list(chain(*all_attention_masks))

total_length = (len(all_input_ids) // block_size) * block_size
all_input_ids = all_input_ids[:total_length]
all_attention_masks = all_attention_masks[:total_length]
print(len(all_input_ids))

In [ ]:
# Split the tokenized texts into chunks of block_size
grouped_input_ids = [all_input_ids[i:i + block_size] for i in range(0, total_length, block_size)]
grouped_attention_masks = [all_attention_masks[i:i + block_size] for i in range(0, total_length, block_size)]
print(f"{len(grouped_input_ids)=}, {len(grouped_attention_masks)=}")

# Create batches of size batch_size
batched_input_ids = [grouped_input_ids[i:i + batch_size] for i in range(0, len(grouped_input_ids), batch_size)]
batched_attention_masks = [grouped_attention_masks[i:i + batch_size] for i in range(0, len(grouped_attention_masks), batch_size)]
print(f"{len(batched_input_ids)=}, {len(batched_attention_masks)=}")

batched_input_ids = [torch.tensor(batch) for batch in batched_input_ids]
batched_attention_masks = [torch.tensor(batch) for batch in batched_attention_masks]
logging.info("Data tokenized and grouped successfully.")

print(f"{batched_input_ids[-1].shape=}, {batched_attention_masks[-1].shape=}")

In [31]:
del vanilla_model
vanilla_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", device_map = 'auto')
logging.info("Vanilla model and tokenizer loaded successfully.")

vanilla_outputs = dict()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from transformer_lens import HookedTransformer
from transformers import AutoModelForCausalLM
tl_model = HookedTransformer.from_pretrained('Llama-2-7b', device="cpu", hf_model=model)
tl_model.eval()


/remote-home/miintern1/anaconda3/envs/mech/lib/python3.12/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025845206/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


NameError: name 'model' is not defined

In [164]:
vanilla_outputs = dict()
def get_layer_output_hook(layer_name):
    def hook_fn(module, input, output):
        # print(f"{output[0].shape=}")
        # print(f"{output[1]=}")
        if layer_name not in vanilla_outputs:
            vanilla_outputs[layer_name] = []
        vanilla_outputs[layer_name].append(output[0].cpu())
    return hook_fn

for i in hook_layer:  # Change the range to match the number of layers you want to capture
    layer_name = f"model.layers.{i}"  # Adjust the layer name/path as needed
    layer = dict([*vanilla_model.named_modules()])[layer_name]
    layer.register_forward_hook(get_layer_output_hook(layer_name))



In [56]:
for key, cache_value in cache.items():
    if(isinstance(cache_value, torch.Tensor)):
        if cache_value.shape == vanilla_outputs['model.layers.31'][0].shape:
            if (torch.allclose(cache_value, vanilla_outputs['model.layers.30'][0], atol=1e-05)):
                print(key)

In [82]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer
model_name = "meta-llama/Llama-2-7b-hf"
vanilla_model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Input text
text = "Please tell me the answer of 1+1="

# Tokenize the input text
inputs = tokenizer(text, return_tensors='pt', truncation=True)
input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']

# Get the logits from the model
output = vanilla_model(input_ids, attention_mask=attention_mask)
logits = output.logits

# Get the predicted token IDs (taking the argmax to get the most likely token at each position)
predicted_token_ids = torch.argmax(logits, dim=-1)

# Decode the predicted token IDs to get the generated text
generated_text = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)

print(generated_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


# note me that truth to the102=?


In [88]:
for i, index in enumerate(input_ids):
    print(tl_logits[0][i][index])

tensor([-0.8467,  1.1656,  1.9924,  3.0213,  5.5284, -0.5593,  4.9252,  8.7389,
         5.5826,  2.1625,  5.5826,  3.0333], grad_fn=<IndexBackward0>)


In [81]:
predicted_token_ids = torch.argmax(output_logit, dim=-1)
generated_text = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)
print(generated_text)

# note me that truth to the102=?


In [75]:
tokenizer.decode(tl_logits[0].argmax(dim=-1).tolist(), skip_special_tokens=True)

'[ help me the difference to this002. ['

In [2]:
text = "Please tell me the answer of 1+1="
inputs = tokenizer(text, return_tensors='pt', truncation=True)
input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
print(input_ids.shape, attention_mask.shape)
output_logit = vanilla_model(input_ids, attention_mask=attention_mask, return_dict=True).logits
tl_logits, cache = tl_model.run_with_cache(input_ids, return_type='logits')
# assert torch.allclose(output_logit, tl_logits[0], atol=1e-05)


NameError: name 'tokenizer' is not defined

In [1]:
tl_logits

NameError: name 'tl_logits' is not defined

In [178]:
final_layer_normalized_result = tl_model.ln_final(cache['blocks.31.hook_resid_post'])
assert (final_layer_normalized_result == cache['ln_final.hook_normalized']).all()
assert (tl_model.unembed(final_layer_normalized_result) == tl_logits).all()

In [160]:
output_logit.shape

torch.Size([1, 12, 32000])

In [166]:
vanilla_outputs['model.layers.31'][0].shape

torch.Size([1, 12, 4096])

In [173]:
torch.softmax(torch.matmul(vanilla_outputs['model.layers.31'][0], model.lm_head.weight.T)[0], dim=-1) - softmax_output_logit
softmax_output_logit

tensor([[4.3590e-10, 1.1438e-07, 1.2302e-04,  ..., 2.0504e-07, 5.8261e-08,
         9.6211e-08],
        [1.1671e-11, 4.1575e-09, 1.3147e-06,  ..., 1.7510e-09, 7.7993e-10,
         6.6821e-09],
        [7.2423e-11, 1.5085e-10, 1.0639e-06,  ..., 3.6436e-09, 2.6215e-09,
         1.9634e-09],
        ...,
        [7.3471e-11, 6.4338e-12, 6.4296e-05,  ..., 9.2317e-10, 3.2505e-09,
         2.3883e-08],
        [1.8955e-10, 4.8383e-10, 1.8349e-02,  ..., 1.2916e-09, 1.0092e-08,
         5.2991e-09],
        [8.9344e-11, 2.0537e-09, 6.7708e-03,  ..., 1.4346e-08, 1.6418e-07,
         1.8087e-08]], grad_fn=<SoftmaxBackward0>)

In [144]:
softmax_tl_logits = torch.softmax(tl_logits[0], dim=-1)
print(softmax_tl_logits.shape)
softmax_output_logit = torch.softmax(output_logit[0], dim=-1)
print(softmax_output_logit.shape)
torch.mean(torch.abs(softmax_tl_logits - softmax_output_logit), dim=-1)

torch.Size([12, 32000])
torch.Size([12, 32000])


tensor([5.7971e-05, 2.3674e-05, 1.9974e-05, 3.2353e-05, 3.3633e-05, 3.8182e-05,
        3.0333e-05, 1.9412e-05, 1.7680e-05, 1.5447e-05, 4.0036e-05, 4.4373e-05],
       grad_fn=<MeanBackward1>)

In [30]:
cache['blocks.11.hook_resid_post'][:, -1, :] == vanilla_outputs['model.layers.31'][0]

tensor([[False, False, False,  ..., False, False, False]])

In [ ]:
device = 'cpu'
vanilla_outputs = dict()
progress_bar = tqdm(total=len(batched_input_ids), desc="Processing batches", leave=True)
i=0
for input_ids, attention_mask in zip(batched_input_ids, batched_attention_masks):
    input_ids = input_ids.to(device)
    print(f"{input_ids.shape=}")
    attention_mask = attention_mask.to(device)
    with torch.no_grad():
        vanilla_output_logit = vanilla_model(input_ids=input_ids, attention_mask=attention_mask).logits
    # del vanilla_output_logit
    torch.cuda.empty_cache()
    progress_bar.update(1)
    break



In [ ]:
vanilla_output_logit[:, -1, :].shape

In [ ]:
watermarked_logits['cygu/llama-2-7b-logit-watermark-distill-kgw-k0-gamma0.25-delta1']['model.layers.28'][0].shape

In [ ]:


class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super(ResidualBlock, self).__init__()
        self.fc = nn.Linear(dim, dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc(x)
        out = self.relu(out)
        out = out + x 
        return out

class TransformModel(nn.Module):
    def __init__(self, num_layers=4, input_dim=1024, hidden_dim=500, output_dim=300):
        super(TransformModel, self).__init__()
        
        self.layers = nn.ModuleList()
        
        self.layers.append(nn.Linear(input_dim, hidden_dim))
        
        for _ in range(num_layers - 2):
            self.layers.append(ResidualBlock(hidden_dim))

        self.layers.append(nn.Linear(hidden_dim, output_dim))

    def forward(self, x):
        for i in range(len(self.layers)):
            x = self.layers[i](x)
        
        return x

# Training function
def train_model(model, dataloader, num_epochs=10, learning_rate=0.001):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for inputs, targets in progress_bar:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            progress_bar.set_postfix(loss=running_loss/len(dataloader))
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

    print("Training complete.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformModel(num_layers=4, input_dim=1024, hidden_dim=500, output_dim=300)
train_model(model, dataloader, num_epochs=10, learning_rate=0.001)


In [ ]:
model_name = 'cygu/llama-2-7b-logit-watermark-distill-aar-k2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
# dataset = load_dataset("allenai/c4", "realnewslike", split="validation", streaming=False)
os.environ['HF_DATASETS_CACHE'] = '/remote-home/miintern1/watermark-learnability/experiments/.cache/huggingface/dataset/'
dataset = load_dataset('allenai/c4', 'realnewslike', split='validation')

In [ ]:
import torch
vanilla_residuals = torch.load("/remote-home/miintern1/watermark-learnability/data/c4/vanilla_residuals.pt", map_location=torch.device('cpu'))
watermarked_residuals = torch.load("/remote-home/miintern1/watermark-learnability/data/c4/watermark_residuals.pt", map_location=torch.device('cpu'))

### Initialize task vector (if needed)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cygu/llama-2-7b-logit-watermark-distill-kgw-k0-gamma0.25-delta1")
watermarked_model = AutoModelForCausalLM.from_pretrained("cygu/llama-2-7b-logit-watermark-distill-kgw-k0-gamma0.25-delta1")
vanilla_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
task_vector = TaskVector(vanilla_model, watermarked_model)

coefficient = 0.1
coefficient_watermarked_model = task_vector.apply_to(vanilla_model, scaling_coef = coefficient)

### Generate Watermarked Samples

In [ ]:
max_length = 250
min_length = 250
num_samples = 512
batch_size = 1
device = 'cpu'
def filter_length(example):
        return len(tokenizer(example['text'], truncation=True, max_length=max_length)["input_ids"]) >= min_length

def encode(examples):
    trunc_tokens = tokenizer(
        examples['text'],
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    ).to(device)
    # Examples are truncated to max_length, which comprises the possible generation prompt and the text to be generated
    examples["text"] = tokenizer.batch_decode(trunc_tokens["input_ids"], skip_special_tokens=True)
    prompt = tokenizer(
        examples["text"], truncation=True, padding=True, max_length=50, return_tensors="pt",
    ).to(device)
    examples["prompt_text"] = tokenizer.batch_decode(prompt["input_ids"], skip_special_tokens=True)
    examples["input_ids"] = prompt["input_ids"]
    examples["attention_mask"] = prompt["attention_mask"]
    examples["text_completion"] = tokenizer.batch_decode(
        trunc_tokens["input_ids"][:, 50:], skip_special_tokens=True
    )
    return examples

dataset = dataset.filter(filter_length)
# Set how many samples will be skipped
dataset = dataset.map(encode, batched=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size)

prompts = []
human_text = []
prompt_text = []
full_human_text = []
for batch in dataloader:
    if len(human_text) >= num_samples:
        break
    if (type(batch["input_ids"]) == list):
        batch["input_ids"] = torch.stack(batch["input_ids"], dim=1).to(device)
    if (type(batch["attention_mask"]) == list):
        batch["attention_mask"] = torch.stack(batch["attention_mask"], dim=1).to(device)
    prompts.append(batch)
    human_text.extend(batch["text_completion"])
    prompt_text.extend(batch["prompt_text"])
    full_human_text.extend(batch["text"])
human_text = human_text[:num_samples]
prompt_text = prompt_text[:num_samples]
full_human_text = full_human_text[:num_samples]
raw_input = {
    "prompts": prompts,
    "human_text": human_text,
    "prompt_text": prompt_text,
    "full_human_text": full_human_text,
}

In [ ]:
input_ids, attention_mask = prompts[0]["input_ids"], prompts[0]["attention_mask"]
print(input_ids.shape, attention_mask.shape)
watermarked_model(input_ids=input_ids, attention_mask=attention_mask).logits

In [ ]:
from tqdm import tqdm
DO_SAMPLE = True
temperature=1.0
top_p=0.9
top_k=0


watermarked_results = []
for batch in tqdm(prompts[:10]):
    with torch.no_grad():
        watermarked_output = watermarked_model.generate(
                            input_ids=batch["input_ids"],
                            attention_mask=batch["attention_mask"],
                            do_sample=DO_SAMPLE,
                            min_new_tokens=200,
                            max_new_tokens=200,
                            temperature=temperature,
                            top_p=top_p,
                            top_k=top_k,
                            pad_token_id=tokenizer.eos_token_id,
                        )
    watermarked_results.append(tokenizer.batch_decode(watermarked_output, skip_special_tokens=True))

In [ ]:
type(watermarked_output)

In [ ]:
from pprint import pprint
pprint(tokenizer.decode(watermarked_output[0], skip_special_tokens=True))
pprint(prompts[0]['text'])

In [ ]:
detector = AarWatermarkDetector(tokenizer=tokenizer, k=2, eps=1e-20)
p_value = detector.detect(tokenizer.decode(watermarked_output[0], skip_special_tokens=True))

In [ ]:
print(p_value)

In [ ]:
watermarked_model = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'cpu')   


In [ ]:
model_name = 'cygu/llama-2-7b-logit-watermark-distill-aar-k3'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
watermarked_model = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'cpu')   


### Watermark Sample Scoring

In [ ]:
watermark_config = data['samples']['llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta1']['watermark_config']

tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
watermark_type = WatermarkType.KGW
detector = WatermarkDetector(
    device=watermark_config.get("kgw_device", 'cpu'),
    tokenizer=tokenizer,
    vocab=tokenizer.get_vocab().values(),
    gamma=watermark_config["gamma"],
    seeding_scheme=watermark_config["seeding_scheme"],
    normalizers=[],
)
model_samples = data['samples']['llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta1']['model_text']
human_samples = data['samples']['llama-2-7b-logit-watermark-distill-kgw-k1-gamma0.25-delta1']['human_text']
model_scores = []
human_scores = []
for sample in tqdm(model_samples):
    score = detector.detect(sample)
    model_scores.append(score['p_value'])
for sample in tqdm(human_samples):
    score = detector.detect(sample)
    human_scores.append(score['p_value'])
    

In [ ]:
import statistics
# Calculate basic statistics
mean = statistics.mean(model_scores)
median = statistics.median(model_scores)
stdev = statistics.stdev(model_scores)
minimum = min(model_scores)
maximum = max(model_scores)

# Print the results
print(f"Mean: {mean}")
print(f"Median: {median}")
print(f"Standard Deviation: {stdev}")
print(f"Minimum: {minimum}")
print(f"Maximum: {maximum}")
